In [2]:
!pip install selenium

In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [47]:
# webdirver 설정, 빅카인즈 사이트 이동
driver = webdriver.Chrome('C:\chromedriver.exe')
driver.implicitly_wait(2)
driver.set_window_size(1500,1200)  #(1280,800)
driver.get('https://www.bigkinds.or.kr/v2/news/index.do')

# 검색기간 1주일로 설정
driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="srch-tab1"]/div/div[1]/span[2]').click()

In [48]:
# 검색어 입력
querytxt = input('크롤링할 키워드는 무엇입니까?:')
searchbox = driver.find_element_by_id("total-search-key")
searchbox.send_keys(querytxt)
searchbox.send_keys("\n")  # 검색버튼 엔터
time.sleep(3)

크롤링할 키워드는 무엇입니까?:오리온


In [49]:
# 기사 분류 -> 경제로 설정
# *주의* 화면을 크게해야 함. 경제 분류가 화면에 보여야 셀레니움이 잡을 수 있음
driver.find_element_by_xpath('//*[@id="filterTab03"]/li[2]/span').click()
time.sleep(3)

# 기사 100건 씩 출력
driver.find_element_by_xpath('//*[@id="select2"]/option[4]').click()
time.sleep(2)

# 전체 페이지 개수
totalPage = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[2]/div/div/div/div/div[3]/div').text
totalPage = int(totalPage)
totalPage

In [53]:
contents = []
texts=[]
curPage = 1

while curPage <= totalPage:
    # bs4 초기화
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # 기사 리스트
    articles = soup.select('div.news-inner')
    # 페이지 번호 출력
    print('Current Page : {}'.format(curPage))

    # 세부 데이터
    for article in articles:
        title = article.select_one('span.title-elipsis').text.strip()
        press =  article.select_one('div.info > div > a').text.strip()
        category = article.select_one('div.info > div > span.bullet-keyword').text.strip()
        date = article.select_one('div.info > p').text.strip()
        contents.append([title, press, category, date])
        
    # 기사 전문 
    for i in range(len(articles)):            
        driver.find_elements_by_css_selector('span.title-elipsis')[i].click()
        time.sleep(1)                
        text = driver.find_elements_by_css_selector('div.news-view-body')[0].text
        texts.append(text.replace('\n', ''))
        
        driver.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()
        time.sleep(1)
            
    # 페이지 수 증가
    curPage += 1
    if curPage > totalPage:
        print('Crawling succeed!')
        break
        
    # 페이지 이동 클릭
    driver.implicitly_wait(3)      
    nextbtn = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
    driver.execute_script("arguments[0].click();", nextbtn)
    
    # bs4 인스턴스 삭제
    del soup
    time.sleep(2)
    
# 브라우저 종료
driver.close() 

len(contents)

----- Current Page : 1 ------
Crawling succeed


44

In [55]:
df = pd.DataFrame(data=contents, columns =['title', 'press', 'category', 'date'])
#df['text'] = texts
# 파일 경로 지정
# df.to_excel("C:/Develops/save/1.xlsx", index=False)
df.to_excel("C:/Develops/finalprac/" + querytxt +".xlsx", index=False)

## 클래스, 함수화

In [14]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options

In [64]:
class CRAWL:
    def __init__(self):
        self.options = Options()
        # headless는 화면이나 페이지 이동을 표시하지 않고 동작하는 모드 (브라우저 창 안보이기)
        #self.options.add_argument('headless')
        self.driver = webdriver.Chrome('C:\chromedriver.exe', options=self.options) 

        
    def getURL(self,url):
        # 빅카인즈 사이트 이동
        self.driver.implicitly_wait(2)
        self.driver.set_window_size(1300,800)
        self.driver.get(url)
        # 검색기간 1주일로 설정
        self.driver.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
        time.sleep(2)
        self.driver.find_element_by_xpath('//*[@id="srch-tab1"]/div/div[1]/span[2]').click()
        time.sleep(2)


    def getPage(self, url, querytxt):
        self.getURL(url)
        # 검색어 입력
        searchbox = self.driver.find_element_by_id("total-search-key")
        searchbox.send_keys(querytxt)  
        searchbox.send_keys("\n")  # 검색버튼 엔터
        time.sleep(2)    
        # 기사 분류 -> 경제로 설정
        self.driver.find_element_by_xpath('//*[@id="filterTab03"]/li[2]/span').click()
        time.sleep(3)
        # 기사 100건 씩 출력
        self.driver.find_element_by_xpath('//*[@id="select2"]/option[4]').click()
        time.sleep(3)
        # 전체 페이지 개수
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        pageNum = soup.select('div.lastNum')[0].text
        self.totalPage  = int(pageNum)
        del soup
        
    
    def crawling(self, url, querytxt):
        self.getPage(url, querytxt)
        print('Total Pages : ', self.totalPage)
        curPage = 1  # 현재 페이지
        self.contents = []
        self.texts = []
        while curPage <= self.totalPage:
            # bs4 초기화
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            # 기사 리스트
            articles = soup.select('div.news-inner')
            # 페이지 번호 출력
            print('Current Page : {}'.format(curPage))
            # 세부 데이터
            for article in articles:
                title = article.select_one('span.title-elipsis').text.strip()
                press =  article.select_one('div.info > div > a.provider').text.strip()
                category = article.select_one('div.info > div > span.bullet-keyword').text.strip()
                date = article.select_one('div.info > p.name').text.strip()
                self.contents.append([title, press, category, date])  
            # 기사 전문 
            for i in range(len(articles)):            
                self.driver.find_elements_by_css_selector('span.title-elipsis')[i].click()
                time.sleep(2)                
                text = self.driver.find_elements_by_css_selector('div.news-view-body')[0].text
                self.texts.append(text.replace('\n', ''))
                self.driver.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()
                time.sleep(2)
            # 페이지 수 증가
            curPage += 1
            if curPage > self.totalPage:
                print('Crawling succeed')
                break    
            # 페이지 이동 클릭
            self.driver.implicitly_wait(3)      
            nextbtn = self.driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
            self.driver.execute_script("arguments[0].click();", nextbtn)
            # bs4 인스턴스 삭제
            del soup
            time.sleep(2)
            
            
    def saving(self, querytxt):
        # 브라우저 종료
        self.driver.close()
        self.df = pd.DataFrame(data=self.contents, columns =['title', 'press', 'category', 'date'])
        self.df['text'] = self.texts
        self.df.to_excel("C:/Develops/finalprac/" + querytxt +".xlsx", index=False)

    

if __name__ == "__main__":
    url = 'https://www.bigkinds.or.kr/v2/news/index.do'
    querytxt = '오리온'
    crawl = CRAWL()
    crawl.crawling(url, querytxt)
    crawl.saving(querytxt)


Total Pages :  1
Current Page : 1
Crawling succeed


+ 최근 1주일
1. 빅카인즈 경제 기사 키워드당 100개만 기져오기
2. 매일경제 + 한국경제 기사 가져오기

In [ ]:
# [제목, 언론사, 분류, 날짜, 기자이름, 기사내용]

# contents = [] 
# for article in articles:
#     title = article.select('span.title-elipsis')
#     press =  article.select('div.info > div > a')
#     category = article.select('div.info > div > span.bullet-keyword')
#     date = article.select('div.info > p')
#     contents.append([title[0].text.strip(), press[0].text.strip(), category[0].text.strip(), date[0].text.strip(), date[1].text.strip()])

# len(contents)

In [ ]:
# 기사 전문 

# texts = []
# for i in range(0, 100):
#     driver.find_elements_by_css_selector('span.title-elipsis')[i].click()
#     time.sleep(1)
#     text = driver.find_elements_by_css_selector('div.news-view-body')[0].text
#     texts.append(text.replace('\n', ''))
#     driver.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()

# len(texts)

In [7]:
# 다음 페이지 넘기기

# nextbtn = driver.find_element_by_xpath('//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a')
# browser.execute_script("arguments[0].click();", nextbtn)
